# Installations

In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

# Download shape_predictor_68_face_landmarks.dat file from dlib

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2024-05-30 14:11:15--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  43.5MB/s    in 1.4s    

2024-05-30 14:11:17 (43.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



# Imports

In [ ]:
import torch
from torchvision import datasets, transforms, utils
import dlib
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import matplotlib.pyplot as plt

# Download LFW dataset

In [ ]:
# Define a transform to convert the images to tensors
transform = transforms.Compose([
    transforms.ToTensor()
])

# Download and load the LFWPeople dataset
lfw_dataset = datasets.LFWPeople(root='./data', split='train', download=True, transform=transform)

# Create a directory to store the landmarks
landmarks_dir = './landmarks'
os.makedirs(landmarks_dir, exist_ok=True)

# Detect facial landmarks and save them

In [ ]:
# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to detect facial landmarks and save them
def detect_landmarks(image, name, idx):
    # Convert tensor image to PIL image
    pil_image = transforms.ToPILImage()(image)

    # Convert PIL image to numpy array
    img_np = np.array(pil_image)

    # Get image dimensions
    img_height, img_width = img_np.shape[:2]

    # Convert image to grayscale
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    # Detect faces in the grayscale image
    faces = detector(gray)

    # If no face detected, return None
    if len(faces) == 0:
        return None

    # Only process the first detected face
    face = faces[0]

    # Predict facial landmarks
    landmarks = predictor(gray, face)

    # Extract landmarks as numpy array (only 0-16 indices)
    landmarks_np = np.array([(landmark.x, landmark.y) for landmark in landmarks.parts()[:17]])

    # Normalize landmarks
    landmarks_normalized = landmarks_np / np.array([img_width, img_height])

    # Create a valid filename by combining the name and index
    valid_filename = f"{name}_{idx}.txt".replace(' ', '_')

    # Save landmarks as scalar values
    np.savetxt(os.path.join(landmarks_dir, valid_filename), landmarks_normalized.flatten())

# Loop through the LFWPeople dataset
for idx, (image, name) in enumerate(lfw_dataset):
    # Detect landmarks and save them
    detect_landmarks(image, name, idx)

print("Landmarks extraction completed!")

In [ ]:
# Download landmark annotations
!zip -r landmarks.zip /content/landmarks